In [65]:
import numpy as np
from PIL import Image
import math
import random

from ray import *
from hittable import *
from camera import *


def degrees_to_radians(doubledegrees):
    return degrees * pi / 180.0;
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
       return v
    return v / norm
def color(array):
    c = array
    c[0] = clamp(c[0], 0, 0.9999)
    c[1] = clamp(c[1], 0, 0.9999)
    c[2] = clamp(c[2], 0, 0.9999)
    c = np.multiply(255.0, c)
    return c
def sample_color(array, samples_per_pixel):

    c = np.multiply(1 / samples_per_pixel, array)
    c[0] = clamp(math.sqrt(c[0]), 0, 0.9999)
    c[1] = clamp(math.sqrt(c[1]), 0, 0.9999)
    c[2] = clamp(math.sqrt(c[2]), 0, 0.9999)
    c = np.multiply(256, c)    
    
    return c

def clamp(x, x_min, x_max):
    if x < x_min :
        return x_min
    if x > x_max :
        return x_max
    return x

def random_in_unit_sphere():
    while(True):
        p = np.random.random(3)
        p = np.multiply(2.0 ,p)
        p = np.subtract(p, np.array([1, 1, 1]))
        if (np.dot(p, p) >= 1):
            continue
        return p

def random_unit_vector():
     return normalize(random_in_unit_sphere())
    


In [66]:
# material defines reflect direction and reflected color.
class material:
    def scatter(r_in, rec, attenuation, scattered):
        pass
class lambertian(material):
    def __init__(self, albedo):
        self.albedo = albedo
    def scatter(r_in, rec, attenuation, scattered):
        scatter_direction = rec.normal + random_unit_vector()
        
        if np.isclose(scatter_direction, np.array([0, 0, 0])):
            scattered_direction = rec.normal
            
        scattered = ray(rec.p, scatter_direction)
        attenuation = self.albedo
        return true

def reflect(v, N):
    return v - (2.0 * np.dot(v, N) * N)
class metal:
    def __init__(self, albedo):
        self.albedo = albedo
    def scatter(r_in, rec, attenuation, scattered):
        reflected = reflect(normalize(r_in.direction), rec.normal)
        scattered = ray(rec.p, reflected)
        attenuation = self.albedo
        return np.dot(scattered.direction, rec.normal) > 0
    

In [67]:
# return the color of a ray
# form: np.array(1, 1, 1)
# world: hittable_list of all objects
def ray_color(r, world, depth):
    rec = hit_record()
    if (depth <= 0):
        return np.array([0, 0, 0])
    
    hit_something, rec = world.hit(r, 0.001, math.inf, rec)
    if (hit_something):
        target = rec.p + random_unit_vector()
        return 0.5 * ray_color(ray.ray(rec.p, target - rec.p), world, depth - 1)
    
#     No Object hit (Background)
    unit_direction = normalize(r.direction)
    t = 0.5 * (unit_direction[1] + 1)
    return (1.0 - t) * np.array([1, 1, 1]) + t * np.array([0.5, 0.7, 1.0])

In [ ]:
# main

# Image
image_width = 160
image_height = 120
aspect_ratio = image_width / image_height
samples_per_pixel = 10
max_depth = 10

# PIL preprocessing to use PIL coordinates
img = Image.new(mode = "RGB", size = (image_width, image_height), color = 0)
img_array = np.array(img)
img_array = np.swapaxes(img_array, 1, 0)



# World object list
world = hittable_list()
world.add(sphere(np.array([0, 0, -1]), 0.5))
world.add(sphere(np.array([0, -100.5, -1]), 100))


#Camera
cam = camera()


# Render
print("image width: ", image_width," image height: ", image_height)
for j in reversed(range(image_height)):
    if j % 50 == 0:
        print(j, "Scanlines remaining")
    for i in range(image_width):
        pixel_color = np.array([0.0, 0.0, 0.0])
        for s in range(samples_per_pixel):
            u = (i + random.random()) / (image_width - 1)
            v = (j + random.random()) / (image_height - 1)
            r = cam.get_ray(u, v)
            pixel_color += ray_color(r, world, max_depth)
        img_array[i][j] = sample_color(pixel_color, samples_per_pixel)

print("Done")

image width:  160  image height:  120
100 Scanlines remaining
50 Scanlines remaining


In [ ]:
final_img_array = np.swapaxes(img_array, 1, 0)
final_img_array = np.flip(final_img_array, 0)
new_pic = Image.fromarray(final_img_array.astype('uint8'), 'RGB')
new_pic.show()